In [23]:
import sys, os,json, cv2

import numpy as np
import glob
from numpy.random import seed
from numpy.random import randint
import joblib
import matplotlib.pyplot as plt, random

import scipy.stats as st

from helper_func import *


path = r'/home/yzstat/yzhan/Maize_images/'
result_path=r'/work/yzstat/yzhan/Maize_images_MobilenetV2_keras/'

seed(42)

# Bagging prediction

In [25]:
test_image_patch_list = sorted(glob.glob(path+ 'maize_image_patches/test/*.png'))
image_list1=np.concatenate(([42], np.linspace(10, 290, 29)))

import tensorflow as tf
# test
lr, batch_size = 0.001, 8
H, W = 256, 256

for n_estimator in [1, 5, 10, 15, 20, 30, 45, 60][:]:
    print('n_estimator', n_estimator)
    out_dir = result_path + 'Multiclass_Prediction/Predict_test_Patches_lr'+\
            str(lr)+'_0.5_batch'+str(batch_size)+'_rs50_750_n_estimator' +\
            str(int(n_estimator))
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
        print('Make directory')
    
    for file in test_image_patch_list[:]:
        aggregated_predictions = np.zeros((H, W, 1), dtype=np.uint8)
        filename = os.path.basename(file)
        for image_seed in np.concatenate((image_list1, np.linspace(300, 330, 31)))[:n_estimator]:
            #pred_test_dir='Multiclass_Prediction/Predict_test_Patches_lr0.001_0.5_batch16_rs50_is42_750'
            pred_test_dir=result_path + 'Multiclass_Prediction/Predict_test_Patches_lr'+\
            str(lr)+'_0.5_batch'+str(batch_size)+'_rs50_is'+\
            str(int(image_seed))+'_750/'
            pred_image = pred_test_dir + filename
            img = tf.io.read_file(pred_image)
            img= tf.image.decode_png(img, channels=1)
            
            aggregated_predictions += img


        aggregated_predictions= np.array(np.round(aggregated_predictions/n_estimator), dtype=np.uint8)
        tf.keras.preprocessing.image.save_img(os.path.join(out_dir, filename), aggregated_predictions, data_format=None, file_format='png', scale=False)


# Bgging IoU


In [ ]:



train_val_size = '750'
lr, batch_size = 0.001, 8
H, W = 256, 256
num_classes=2

# Test IoU
for n_estimator in [1, 5, 10, 15, 20, 30, 45, 60][:]:
    print('n_estimator', n_estimator)
    pred_test_dir ='Multiclass_Prediction/Predict_test_Patches_lr'+\
            str(lr)+'_0.5_batch'+str(batch_size)+'_rs50_750_n_estimator' +\
            str(int(n_estimator))
    out_dir = result_path+ 'multiclass/iou_pkl_dropout_lr' + \
            str(lr)+'_0.5_batch' + str(batch_size)+ '_rs50_n_estimator'  +\
            str(int(n_estimator))
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
        print('Make directory')
    test_image_patch_list = sorted(glob.glob(path+ r'maize_image_patches/test/*.png'))
    mgray_list=[]

    iou=[]
    intersection, union = np.zeros(num_classes), np.zeros(num_classes)
    all_mask, all_pred =np.array([]), np.array([])
    inter_list, uni_list=[], []


    pred_test_mask_patch_list=sorted(glob.glob(os.path.join(result_path+pred_test_dir, '*.png')))

    test_mask_patch_list=sorted(glob.glob(path+'maize_label_patches/test/*.png'))

    test_image_patch_files=[os.path.basename(file) for file in test_image_patch_list]

    pred_test_mask_patch_files=[os.path.basename(file) for file in pred_test_mask_patch_list]
    test_mask_patch_files=[os.path.basename(file) for file in test_mask_patch_list]
    assert pred_test_mask_patch_files==test_mask_patch_files, 'pred, ground truth do not match.'
    assert test_image_patch_files==test_mask_patch_files, 'image, mask do not match.'

    print('pred test image size: ', len(pred_test_mask_patch_list))


    for i in range(len(test_mask_patch_list)):
        #print(i)
        #assert os.path.basename(test_mask_patch_list[i])==os.path.basename(pred_mask_patch_list[i])

        img=cv2.imread(test_image_patch_list[i], 0)
        mgray_list.append(np.mean(img))

        mask = tf.io.read_file(test_mask_patch_list[i])
        mask= tf.image.decode_png(mask, channels=1)

        y_pred=tf.io.read_file(pred_test_mask_patch_list[i])
        y_pred= tf.image.decode_png(y_pred, channels=1)

        iou.append(iou_dice(y_pred, mask, num_classes))

        inter, uni=intersection_union(y_pred, mask, num_classes)
        intersection, union=intersection+inter, union+uni

        inter_list.append(inter)
        uni_list.append(uni)

        all_mask=np.append(all_mask, tf.reshape(mask, [-1]))
        all_pred=np.append(all_pred, tf.reshape(y_pred, [-1]))


    joblib.dump(iou, os.path.join(out_dir, 'test_' + train_val_size+'.pkl' ))
    joblib.dump(mgray_list, os.path.join(out_dir, 'test_grayness.pkl' ))
    gen_iou=intersection/union
    joblib.dump(gen_iou, os.path.join(out_dir, 'test_gen_' + train_val_size+'.pkl' ))

    joblib.dump(inter_list, os.path.join(out_dir, 'test_interaction_' + train_val_size+'.pkl' ))
    joblib.dump(uni_list, os.path.join(out_dir, 'test_union_' + train_val_size+'.pkl' ))

    cm_analysis(all_mask, all_pred, os.path.join(out_dir, 'test_cm_' + train_val_size+'.png'), save_file=True )


In [ ]:
all_random_iou, all_random_iou2, all_random_miou, all_random_miou2=[], [], [], []

lr, batch_size = 0.001, 8
H, W = 256, 256
x=[1, 5, 10, 15, 20, 30, 45, 60]
#x=[str(int(xi)) for xi in x]
print(x)
size = 13
start, end=0, len(x)

#plt.subplots(figsize=(16, 4))
fig, axs = plt.subplots(1, 3, figsize=(14,4), sharey=True)

left  = 0.1  # the left side of the subplots of the figure
right = 0.9    # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.3   # the amount of width reserved for blank space between subplots
hspace = 0.2   # the amount of height reserved for white space between subplots


plt.subplots_adjust(left=left,
                    bottom=bottom, 
                    right=right, 
                    top=top, 
                    wspace=wspace, 
                    hspace=hspace)



random_iou, random_mean_iou, = [], []

# Test IoU
for n_estimator in [1, 5, 10, 15, 20, 30, 45, 60][:]:
    #r'multiclass/dropout_rate_active_learning_lr' + str(lr) + '_' +\
#    str(dropout_rate)+'_batch' + str(batch_size)+\
#'_select_ratio'+str(select_ratio)+'_rs'+str(seed)+'_is'+str(image_seed)
    #iou_pkl_dropout_lr0.001_0.5_batch16_rs50_is42
    
    
    iou_files=sorted(glob.glob(result_path+'multiclass/iou_pkl_dropout_lr'+str(lr)+\
                               '_0.5_batch'+str(batch_size)+'_rs50_n_estimator'  +\
                                str(int(n_estimator))+
                              '//test_750.pkl'),\
                     key = lambda x: int(os.path.basename(x).split('_')[-1].split('.')[0])) 
    #print(iou_files)
   
    
    for file in iou_files[:]:
        
        #print(os.path.basename(file))
        filename=os.path.basename(file)


        #print(os.path.basename(file))
        if image_seed ==42: 
            print('random')
            print(os.path.basename(file))

        iou=joblib.load(file) #all image patch iou of one model prediction
        #print(file)
        
        iou=np.nanmean(iou, axis=0) #dim:2, Iou0, iou for that one model prediction
        #print(iou)
        random_iou.append(iou)

        mean_iou=np.nanmean(iou) #mean iou: miou
        random_mean_iou.append(mean_iou)
        miou=np.mean(mean_iou)

        



for i in range(2):

    axs[i].yaxis.set_tick_params(labelbottom=True, labelsize=size)
    axs[i].plot(x[start: end], np.array(random_iou)[start: end, i], label='random_lr'+\
                str(lr)+'_batch'+str(batch_size))
    
    axs[i].set_ylabel('IoU$_{}$'.format(i), fontsize = size )
    axs[i].set_xlabel('number of estimators', fontsize=size)
    axs[i].set_xticks(x[start: end])
    axs[i].set_xticklabels(x[start: end], rotation =90,fontsize=size)

    axs[i].set_ylim(0.62, 0.73)

    #plt.show()

#plt.subplot(155)
axs[-1].yaxis.set_tick_params(labelbottom=True, labelsize=size)
axs[-1].plot(x[start: end], np.array(random_mean_iou)[start: end])#, label='random_lr'+str(lr)+'_batch'+str(batch_size))


   
axs[-1].set_ylabel('mIoU', fontsize = size )
axs[-1].set_xlabel('sample size', fontsize=size)
axs[-1].set_xticks(x[start: end])
axs[-1].set_xticklabels(x[start: end], rotation =90, fontsize=size)
plt.savefig(result_path+'/multiclass/bagging.png', bbox_inches='tight', dpi=600)
#plt.savefig(result_path+'/Multiclass/visualization/test_iou_full.png', bbox_inches='tight', dpi=600)
plt.show()

